In [25]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
le = LabelEncoder()

In [22]:
def label_encode_dataset(dataset):
    for column in dataset.columns.values:
        dataset[column] = dataset[column].astype(str)
        le.fit(y=dataset[column])
        dataset[column] = le.transform(dataset[column])
    return dataset

In [46]:
#import the data I need
test_data = pd.read_csv('csv_files/test.csv')
animal_data = pd.read_csv('csv_files/train.csv')

#Drop columns that I don't need
animal_data = animal_data.drop(labels=['AnimalID', 'Name', 'DateTime', 'OutcomeSubtype'], axis=1)

#setting up the test data
test_index = test_data.ID
test_data = test_data.drop(labels=['ID', 'Name', 'DateTime'], axis=1)
test_data = label_encode_dataset(test_data)

#splitting up the training data so I can use it to test how well my predictions are coming along
train_animal_data, test_animal_data = train_test_split(animal_data, test_size= .25, random_state=53)

#since the evaluation for this kaggle competition will be in multi-class logloss I've popped the outcomes off of the
#dataset and changed the results into dummy variables
outcome_train_animal_data = train_animal_data.pop('OutcomeType')
le.fit(outcome_train_animal_data)
outcome_train_animal_data = le.transform(outcome_train_animal_data)

#repeat for the test data
outcome_test_animal_data = test_animal_data.pop('OutcomeType')
le.fit(outcome_test_animal_data)
outcome_test_animal_data = le.transform(outcome_test_animal_data)

train_animal_data = label_encode_dataset(train_animal_data)
test_animal_data = label_encode_dataset(test_animal_data)

/Users/MatthewBarnette/shelter_animals/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/MatthewBarnette/shelter_animals/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [63]:
dtrain = xgb.DMatrix(train_animal_data, label=outcome_train_animal_data)
dtest = xgb.DMatrix(test_animal_data, label=outcome_test_animal_data)

In [134]:
parm = {'bst:max_depth':1, 'bst:eta':1, 'silent':1, 'objective':'multi:softprob', 'num_class':5, 'max_delta_step':1}
parm['nthread'] = 1
parm['eval_metric'] = 'mlogloss'

In [135]:
evallist = [(dtrain,'train'), (dtest,'eval')]

In [136]:
num_round = 100
bst = xgb.train(parm, dtrain, num_round, evallist, early_stopping_rounds=3)

Will train until eval error hasn't decreased in 3 rounds.
[0]	train-mlogloss:1.182768	eval-mlogloss:1.191795
[1]	train-mlogloss:1.073994	eval-mlogloss:1.086185
[2]	train-mlogloss:1.030188	eval-mlogloss:1.045041
[3]	train-mlogloss:1.008730	eval-mlogloss:1.026673
[4]	train-mlogloss:0.994793	eval-mlogloss:1.012845
[5]	train-mlogloss:0.985380	eval-mlogloss:1.005189
[6]	train-mlogloss:0.978674	eval-mlogloss:0.999544
[7]	train-mlogloss:0.972606	eval-mlogloss:0.993433
[8]	train-mlogloss:0.968068	eval-mlogloss:0.988502
[9]	train-mlogloss:0.964351	eval-mlogloss:0.984436
[10]	train-mlogloss:0.960964	eval-mlogloss:0.981067
[11]	train-mlogloss:0.957932	eval-mlogloss:0.979344
[12]	train-mlogloss:0.955571	eval-mlogloss:0.977957
[13]	train-mlogloss:0.953208	eval-mlogloss:0.977293
[14]	train-mlogloss:0.951341	eval-mlogloss:0.975390
[15]	train-mlogloss:0.949385	eval-mlogloss:0.973697
[16]	train-mlogloss:0.947218	eval-mlogloss:0.970894
[17]	train-mlogloss:0.945588	eval-mlogloss:0.969952
[18]	train-mlogl

In [137]:
dtest = xgb.DMatrix(test_data)
ypred = bst.predict(dtest, output_margin=False)

/Users/MatthewBarnette/shelter_animals/.direnv/python-3.5.0/lib/python3.5/site-packages/xgboost/core.py:840: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  preds = preds.reshape(nrow, preds.size / nrow)


In [139]:
pd.DataFrame(ypred)

,0,1,2,3,4
0,0.046518,0.010577,0.076670,0.148234,0.718000
1,0.526294,0.002603,0.025972,0.235838,0.209295
2,0.607212,0.007240,0.032945,0.057080,0.295523
3,0.036110,0.011053,0.105230,0.240750,0.606858
4,0.482738,0.001550,0.027143,0.280336,0.208234
5,0.460087,0.002271,0.031072,0.290350,0.216220
6,0.581632,0.007598,0.081169,0.108504,0.221097
7,0.590968,0.007889,0.046457,0.087922,0.266763
8,0.503792,0.001672,0.026269,0.264530,0.203738
9,0.545966,0.001734,0.025998,0.244781,0.181521
